In [12]:
%cd /home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen/

import numpy as np;
from embeddings.h5_processor import H5DataProcessor, DatasetConfig, ProcessedDataset, AudioSegmentSplitter;
from sklearn.preprocessing import StandardScaler;
from sklearn.model_selection import train_test_split
import pandas as pd;
from universal_music.models import RatingsClassifier;

/home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen


In [10]:
# Process datasets using H5DataProcessor
processor = H5DataProcessor()

config = {
    'dataset': 'embeddings/NHS',
    'subfolder': 'samples/wav'
}

dataset = processor.process_h5_file(
    processor.get_embedding_path(DatasetConfig(**config)),
    DatasetConfig(**config)
)


In [3]:
embeddings = dataset.embeddings
filenames = dataset.filenames

df = pd.read_csv('universal_music/FFfull.csv', low_memory=False)

web_df = df[df['study'] == 'web'].copy()
field_df = df[df['study'] == 'field'].copy()

song_functions = df.groupby("song").agg({'songfunction': 'first'})

# Get ratings and normalize
web_ratings = web_df[["song","danc","heal","baby","love"]]

In [4]:
import numpy as np
import pandas as pd
import re

# Function to extract song ID from filename
def extract_song_id(filename):
    # Extract the base filename
    base_name = filename.split('/')[-1]
    # Extract numbers from filename
    match = re.search(r'(\d+)', base_name)
    if match:
        return int(match.group(1))
    return None

# Create a mapping from filename to embedding index
filename_to_idx = {name: idx for idx, name in enumerate(filenames)}

# Extract song IDs from filenames
song_ids = [extract_song_id(filename) for filename in filenames]

# Create a mapping from song ID to embedding index
song_id_to_idx = {song_id: idx for idx, song_id in enumerate(song_ids) if song_id is not None}

# Get all ratings (each row is a rating by one participant for one song)
ratings_df = web_df[["song", "danc", "heal", "baby", "love"]]

# Create X and y matrices
X_list = []
y_list = []

# For each rating entry
for _, row in ratings_df.iterrows():
    song_id = row['song']
    
    # Check if we have an embedding for this song
    if song_id in song_id_to_idx:
        embedding_idx = song_id_to_idx[song_id]
        embedding = embeddings[embedding_idx]
        
        # Add this embedding to X
        X_list.append(embedding)
        
        # Add corresponding ratings to y
        y_list.append([
            row['danc'],
            row['heal'],
            row['baby'],
            row['love']
        ])

# Convert lists to arrays
X = np.array(X_list)
y = np.array(y_list)

In [16]:
# Create and configure model
model = RatingsClassifier(max_iter=10)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
# Training
model.fit(X_train_scaled, y_train, verbose=True)

Loss: 5.7236: 100%|██████████| 10/10 [00:28<00:00,  2.85s/it]


In [18]:
# Predicting ratings on the test set
pred_ratings = model.predict_ratings(X_test_scaled)